# README

In questo file Jupyter è stato implementato il Cloud Centrale, di capacità infinita, sia per la prima che per la seconda euristica.

Per eseguire il codice bisogna usare le funzioni **run_heuristic_nodo_centrale** per la prima euristica, **multi_agente_nodo_centrale** per la seconda.

- a) **Parametri da passare alla prima euristica:**
     - 1) lista di richieste (che può variare ad ogni ciclo)
     - 2) lista di capacità (una volta fissata, resta costante per tutte le iterazioni)
     - 3) numero di nodi all'interno del sistema di fog computing
     - 4) un contatore di scambi
     - 5) lista del costo di rigetto dei nodi (una volta fissata, anch'essa rimane fissa)
     - 6) costo del cloud (fisso)

- b) **Parametri da passare alla seconda euristica:**
     - 1) lista della differenza tra richieste e capacità
     - 2) lista di richieste (che può variare ad ogni ciclo)
     - 3) lista di capacità (una volta fissata, resta costante per tutte le iterazioni)
     - 4) numero di nodi all'interno del sistema di fog computing
     - 5) un contatore di scambi
     - 6) lista del costo di rigetto dei nodi (una volta fissata, anch'essa rimane fissa)
     - 7) costo del cloud (fisso)

**NB**:
- 1) La lista di richieste deve essere estratta da una distribuzione di Poisson che abbia come media la media della capacità totale del sistema (capacità della cpu e capacità della ram). Il secondo paramentro è il numero di nodi.
- 2) La lista della capacità viene generata randomicamente in un range che va da 2 a 100 (2 è la richiesta di ram e cpu per ogni richiesta che viene fatta al sistema)
- 3) Il costo di rigetto dei nodi viene estratto anch'esso da una distribuzione di Poisson, che ha come media il costo del cloud.

**NB 2**:
Il codice può essere eseguito anche senza iterazioni, ma, per una migliore simulazione, si consiglia di farle.

In [ ]:
#vengono importate le librerie necessarie
import random
import time
import numpy as np

Vengono di seguito definite le funzioni che serviranno poi per la costruzione della prima euristica *run_heuristic_nodo_centrale*

In [56]:
def diff(request,capacity,n):
    difference=[]
    for i in range(0,n):
        difference.append(capacity[i]-request[i])
    
    difference=sorted(difference,reverse=True)
        
    return difference

def iterate(difference,n,count):
    i=0
    while(difference[n-1]<0):
        difference[n-1]=difference[n-1] + difference[i]
        difference[i]=0
        i=i+1
        count+=1
    return difference,count


def fog_computing(difference,n,count):
    if(difference[n-1]>=0):
        print("No changing required:",difference)
    else:
        while(difference[n-1]<0):
            if(abs(difference[n-1]) <= abs(difference[0])):
                difference[0]=difference[0] + difference[n-1]
                difference[n-1]=0
                count+=1
            else:
                difference,count=iterate(difference,n,count)
            
            difference=sorted(difference,reverse=True)
        
        print("System completed:",difference)
        print("Number of changes:",count)
        
def run_heuristic_nodo_centrale(request,capacity,n,count, costo_rigetto_nodi,costo_cloud):
    tot_request=sum(request)
    tot_capacity=sum(capacity)
    if(tot_request > tot_capacity):
        print("System overloaded")
        dif_tot=tot_request-tot_capacity
        if(costo_rigetto_nodi[costo_rigetto_nodi.index(max(costo_rigetto_nodi))]> costo_cloud):
            print("Le {} richieste vanno nel cloud".format(dif_tot))
            while(dif_tot>0):
                max_rigetto=costo_rigetto_nodi.index(max(costo_rigetto_nodi))
                if(request[max_rigetto]-dif_tot>=0):
                    request[max_rigetto]=request[max_rigetto]-dif_tot
                    dif_tot=0
                    count+=1
                else:
                    dif_tot=dif_tot-request[max_rigetto]
                    request[max_rigetto]=0
                    costo_rigetto_nodi[max_rigetto]=0
                    count+=1
                    #print(dif_tot,request,costo_rigetto_nodi)


        
            print("Starting the process...")
            differenza=diff(request,capacity,n)
            fog_computing(differenza,n,count)
        else:
            print("Le {} richieste vengono rigettate".format(dif_tot))
            i=0
            while(dif_tot>0):
                request[i]=request[i]-1
                dif_tot=dif_tot-1
                i=i+1
                if(i==n-1):
                    i=0
        
            print("Starting the process...")
            differenza=diff(request,capacity,n)
            fog_computing(differenza,n,count)
    else:
        print("Starting the process...")
        differenza=diff(request,capacity,n)
        fog_computing(differenza,n,count)


Vengono di seguito definite le funzioni che serviranno poi per la costruzione della seconda euristica *multi_agente_nodo_centrale*

In [57]:
def diff_multi_agente(request,capacity,n):
    difference=[]
    for i in range(0,n):
        difference.append(capacity[i]-request[i])
    return difference

def multi_agente_nodo_centrale(differenza, requests, n_richieste, m, scambi,costo_nodi,costo_cloud):
    indici=[]
    values=[]

    for i in range(len(differenza)):
        if(differenza[i]<0):
            indici.append(i)
            values.append(differenza[i])

    #print('Indici:',indici,'Valori:',values)  
    for i in indici:
        #print("Nodo:{}".format(i))        
        for j in range(len(differenza)):
            if(differenza[j]>=abs(values[indici.index(i)]) and j!=i):
                differenza[j]=differenza[j]+values[indici.index(i)]
                values[indici.index(i)]=0
                differenza[i]=0
                scambi+=1
                #print("Svuotamento completato")
                #print(differenza,values)
                break

        if(values[indici.index(i)]!=0):
            #print('Nodo {} da svuotare in splitting:'.format(i))
            #print(differenza,values)
            while(values[indici.index(i)] < 0):
                if(sum(differenza)>= 0): 
                    for k in range(len(differenza)):
                        if(differenza[k]>0):
                            if(abs(values[indici.index(i)])>differenza[k]):
                                values[indici.index(i)] = values[indici.index(i)] + differenza[k]
                                differenza[k]=0
                                scambi+=1
                            else:
                                differenza[k]=differenza[k]+values[indici.index(i)]
                                values[indici.index(i)]=0
                                scambi+=1
                            #print(differenza,values)
                else:
                    print('System overloaded')
                    if(abs(sum(differenza)) > abs(differenza[i])):
                        if(costo_nodi[i]>costo_cloud):
                            print("Le {} richieste vengono inviate al cloud".format(abs(differenza[i])))
                            scambi+=1
                            #print('{} richieste rigettate'.format(abs(differenza[i])))
                        else:
                            print(' Le {} richieste vengono rigettate'.format(abs(differenza[i])))
                        
                        values[indici.index(i)] = 0
                        differenza[i] = 0
                    else:
                        if(costo_nodi[i]>costo_cloud):
                            print("Le {} richieste vengono inviate al cloud".format(abs(sum(differenza))))
                            scambi+=1
                        else:
                            print('Le {} richieste vengono rigettate'.format(abs(sum(differenza))))
                        
                        values[indici.index(i)] = values[indici.index(i)] - sum(differenza)
                        differenza[i] = differenza[i] - sum(differenza)          
            differenza[i]=0
            #print(differenza,values)
            #print('Nodo svuotato')
            
    print('Numero scambi seconda euristica:',scambi)
    return scambi

Di seguito vengono eseguite 10 iterazioni sia sulla prima che sulla seconda euristica, dove il numero di nodi viene settato a 6 (*m=6*), il costo del cloud viene fissato a 10 (*costo_cloud=10*), le richieste di ram e cpu vengono fissate a 2 (*ram_richiesta=2* e *cpu_richiesta=2*), le capacità di ram e cpu vengono settate randomicamente per gli m nodi, in un range che varia tra il numero di richiesta ram (o cpu) e 100.
Infine, il numero di richieste viene estratto da una distribuzione di Poisson, che ha come media la media della capacità totale del sistema (ovvero capacità della cpu e della ram).
Il costo di rigetto dei nodi viene anch'esso estratto da una distribuzione di Poisson, che ha come media il costo del cloud.

In [77]:
for iter in range(0,10):
    print("ITERATION {}".format(iter+1))
    m=6
    costo_cloud=10
    costo_rigetto_nodi=np.random.poisson(costo_cloud,m).tolist()
    ram_richiesta = 2
    cpu_richiesta = 2
    ram_capacity=random.sample(range(ram_richiesta,100),m)
    cpu_capacity=random.sample(range(cpu_richiesta,100),m)
    capacity_tot = []
    for i in range(0,m): 
        capacity_tot.append([ram_capacity[i],cpu_capacity[i]])
    n_richieste = [min(x) / ram_richiesta for x in capacity_tot]
    n_richieste = [int(x) for x in n_richieste]
    media_totale = np.mean(n_richieste).tolist()

    richieste=np.random.poisson(media_totale-0.5,m)

    differenza=diff_multi_agente(richieste,n_richieste,m)

    scambi=0
    count=0

    print("Richieste disponibili per nodo:",n_richieste)
    print("Richieste in entrata:",richieste)

    run_heuristic_nodo_centrale(richieste,n_richieste,m,count,costo_rigetto_nodi,costo_cloud)

    multi_agente_nodo_centrale(differenza, richieste, n_richieste, m, scambi,costo_rigetto_nodi,
                              costo_cloud)

ITERATION 1
Richieste disponibili per nodo: [11, 8, 30, 4, 42, 15]
Richieste in entrata: [16 21 20 22 19 16]
System overloaded
Le 4 richieste vanno nel cloud
Starting the process...
System completed: [0, 0, 0, 0, 0, 0]
Number of changes: 6
System overloaded
Le 4 richieste vengono inviate al cloud
Numero scambi seconda euristica: 6
ITERATION 2
Richieste disponibili per nodo: [8, 1, 12, 29, 3, 1]
Richieste in entrata: [ 7 10 11 10 11  6]
System overloaded
Le 1 richieste vanno nel cloud
Starting the process...
System completed: [0, 0, 0, 0, 0, 0]
Number of changes: 6
System overloaded
Le 1 richieste vengono rigettate
Numero scambi seconda euristica: 5
ITERATION 3
Richieste disponibili per nodo: [3, 27, 4, 38, 6, 1]
Richieste in entrata: [12 12 12 12  9 19]
Starting the process...
System completed: [3, 0, 0, 0, 0, 0]
Number of changes: 4
Numero scambi seconda euristica: 4
ITERATION 4
Richieste disponibili per nodo: [8, 6, 12, 9, 20, 2]
Richieste in entrata: [ 8 10  9 10  8  8]
Starting the